# Parsing semi-structured data

## Intro to TextFSM

The TextFSM library is a parsing library that allows you to conveniently parse
semi-structured data.

When dealing with various network and system admin tasks with things that do
NOT have programmatic APIs you'll often find that you get structured text
outputs that you want to parse. This library is useful for parsing such data.

Install with:

In [ ]:
pip install textfsm

The FSM in the name TextFSM refers to a Finite State Machine, this is a
conceptual model of computation that the TextFSM library is built on top of.

### The overall goal

The goal when using `textfsm` is to get the information from devices into our
Python scripts in such a way that we can easily automate things. We want to be
able to get information like this:

In [ ]:
Interface                Status         Protocol Description
Fa1                      admin down     down
Te1/1                    up             up       lexample/Eth2/22
Te1/2                    up             up       oexample/Eth2/22

into Python like this:

In [ ]:
['Fa1','admin down','down','']
['Te1/1','up','up','lexample/Eth2/22']
['Te1/2','up','up','oexample/Eth2/22']

### When you might want to use this library

If you have some complicated tabular data without an API to interact with it,
you may consider parsing the textual output directly.

If you have some tabular data that has some information in it that you wish to
extract, as long as only one entry per row matches a specific textual format
you could go about this task in many ways.  One such way is to use a regular
expression. For some simple tasks this gets the job done, but there are issues
that come up with more complex tasks.

One of the most annoying situations is when you have free-form comments or rows
with optional output since this can make parsing considerably harder. We can
see some of these issues in the following example:

In [1]:
! python3 extras/phone_project.py

number       status       project  description
0491 570 157 active       A        used to be in project B
0491 570 158 active       A        
0491 570 159 disconnected A        
0491 570 110 active       B        was disconnected on 2019/12/01 activated again 2019/12/02
0491 570 313 active       B        previously was 0491 573 087
0491 570 737 inactive     B        Deactivated 2019/06/09
0491 571 266 disconnected C        
0491 571 491 disconnected C        
0491 571 804 active       C        
0491 572 549 active       C        
0491 572 665 active       Update   
0491 572 983 inactive     Update   Deactivated 2019/06/01
0491 573 770 disconnected Update   
0491 573 087 disconnected          


A non-contrived example is the Cisco `show interfaces description` where one of
the output columns in a free form text description that people can enter into
the devices.

You could try to parse some text like this by splitting each row based on
whitespace and positions. For example here you have the option to split each
row when you see two or more spaces. Having that free-form "description" column
can make this hard to do depending on what is in it, as a user could enter in
more than one space by accident for example.

Or you could put together a regex to search for particular terms. This would
have the downside of perhaps matching things in the wrong column. Once again
this description column can be annoying; we would need to have some logic to
make sure that if we matched a value for a phone number it would only be at the
start of the line.  Including this context starts to make the regex more
complicated to write and more prone to errors.

TextFSM allows you to nicely combine the good parts of both the regex approach
and the positional parsing, which gives you substantially more power to parse
the text.

For tabular structured text data TextFSM is often more convenient than having
to roll out your own fully customized parser using something like PyParsing or
LR parser or some other parser generator.

## TextFSM templates

To use TextFSM you will need to first create a template that defines the
structure of the expected output.

It is by defining this structure that you get the power from a parser like
TextFSM.

A TextFSM template consists of:

1. Value Definitions
2. State Definitions
3. Actions

Values are the smallest building block and will allow you to specify a value
you may find in a record, this is similar to defining the contents of a cell in
a tabular data format.  States are a groups of values according to a specific
format, these specify valid rows.  Actions lets you define what happens when
you find a match of a State.

### Value definitions

These specify what can be in an entry of data, usually representing a column,
these go at the top of a template.

To define a value we have to use the `Value` keyword, followed by the name,
followed by a regex that must be matched for extracting that column.

For example:

In [ ]:
Value PORT (\S+)
Value STATUS (up|down|admin\s+down)
Value PROTOCOL (up|down)
Value DESCRIP (\S.*?)

Values are like a cell in a tabular data format. Much like regex capture groups
each value is used later to map to an item in the parsed output.

### State definitions

A state is effectively a way of defining what makes up an item which you want
to extract.

To define a state we have to create a pattern that will match the state.
Because we are dealing with textual input, a state can be spread across more
than one row of text.

We define a rule per line of text for each state:

In [ ]:
stateName
  ^rule1
  ^rule2
  ...
  ^ruleN

Where each rule is comprised of a pattern that includes whatever combination of
regexes and `Value`s we need.

Note: each line under a state name must start with exactly 2 spaces!

### Actions

Once you have defined `Value`s and how they combine into `State`s you have to
specify how they transfer to other states.

There are some special predefined `Action`s that will do useful things:

- `Record` will store this state in a results record
- `Error` will bail out the whole parsing process and discard all results so far

When you have a match for a state you then have to decide how to handle that
match, this is what TextFSM refers to as an action.

The most common `Action` we will use is `Record` which tells TextFSM to store a
match in a record. The next most common `Action` is to transfer to another
`State`; this is useful for some more complex parsing tasks where you may have
something like nested data.

## Example: a simple template

Let's parse the following text output into a nice Python data structure:

In [2]:
! python3 extras/phone_project.py --simple

number       status      
0491 570 157 active      
0491 570 158 active      
0491 570 159 disconnected
0491 570 110 active      
0491 570 313 active      
0491 570 737 inactive    
0491 571 266 disconnected
0491 571 491 disconnected
0491 571 804 active      
0491 572 549 active      
0491 572 665 active      
0491 572 983 inactive    
0491 573 770 disconnected
0491 573 087 disconnected


Note that there is one record per line in the data being output here.

In [3]:
%%writefile phone_simple.textfsm
Value PhoneNumber (\d+ \d+ \d+)
Value Status (\S+)

Start
  ^${PhoneNumber}\s+${Status} -> Record

Overwriting phone_simple.textfsm


In [4]:
raw_text_data = """number       status      
0491 570 157 active      
0491 570 158 active      
0491 570 159 disconnected
0491 570 110 active      
0491 570 313 active      
0491 570 737 inactive    
0491 571 266 disconnected
0491 571 491 disconnected
0491 571 804 active      
0491 572 549 active      
0491 572 665 active      
0491 572 983 inactive    
0491 573 770 disconnected
0491 573 087 disconnected"""

import textfsm

# Load in the TextFSM template
with open("extras/phone_simple.textfsm", mode='rt') as template:
    re_table = textfsm.TextFSM(template)
fsm_results = re_table.ParseText(raw_text_data)
fsm_results

[['0491 570 157', 'active'],
 ['0491 570 158', 'active'],
 ['0491 570 159', 'disconnected'],
 ['0491 570 110', 'active'],
 ['0491 570 313', 'active'],
 ['0491 570 737', 'inactive'],
 ['0491 571 266', 'disconnected'],
 ['0491 571 491', 'disconnected'],
 ['0491 571 804', 'active'],
 ['0491 572 549', 'active'],
 ['0491 572 665', 'active'],
 ['0491 572 983', 'inactive'],
 ['0491 573 770', 'disconnected'],
 ['0491 573 087', 'disconnected']]

### A potential gotcha with matching end of lines

One thing you may have been tempted to do in the last example is to create a state that has to match the end of the row using `$$`, for example:

In [ ]:
Start
  ^${PhoneNumber}\s+${Status}$$ -> Record

However, this can cause problems since trailing whitespace after the end of the
shorter rows will NOT match if we explicitly state the `Status` ends the row
exactly. Keep trailing whitespace in mind if you get missing matches when using
`$$` for ends of row matching.

There are times where `$$` is very useful. For example, with free-form entries like
descriptions it can be very good to match the end of the line explicitly, since
these columns can be arbitrary in length and contents but always end at the end
of the line.

## Exercise: parsing semi-structured text

Create a template to parse the original example text table:

In [5]:
!python3 extras/phone_project.py

number       status       project  description
0491 570 157 active       A        used to be in project B
0491 570 158 active       A        
0491 570 159 disconnected A        
0491 570 110 active       B        was disconnected on 2019/12/01 activated again 2019/12/02
0491 570 313 active       B        previously was 0491 573 087
0491 570 737 inactive     B        Deactivated 2019/06/09
0491 571 266 disconnected C        
0491 571 491 disconnected C        
0491 571 804 active       C        
0491 572 549 active       C        
0491 572 665 active       Update   
0491 572 983 inactive     Update   Deactivated 2019/06/01
0491 573 770 disconnected Update   
0491 573 087 disconnected          


We have 4 columns so we need 4 `Value` definitions. The first 3 are required and the last is optional.

Create a TextFSM template that will extract the data from this formatted output.

Hint the free-form description column can use a regex like `(\S.*?)`

In [ ]:
# See solutions/parse_phone_projects_table.py

## Storing output in a Pandas DataFrame

If you want to do some sort of analysis on these outputs it can be useful to
store the results in a Pandas DataFrame.

To do this we have to create new DataFrame with the right column names then
append to it as the rows come in from TextFSM.

In [10]:
raw_data_all = """number         status        project   description
0491 570 157   active        A         used to be in project B
0491 570 158   active        A
0491 570 159   disconnected  A
0491 570 110   active        B         was disconnected on 2019/12/01 activated again 2019/12/02
0491 570 313   active        B         previously was 0491 573 087 
0491 570 737   active        B
0491 571 266   disconnected  C
"""

# Load in the TextFSM template
with open("solutions/phone_info_all.textfsm", mode='rt') as template:
    re_table = textfsm.TextFSM(template)
fsm_results = re_table.ParseText(raw_data_all)

import pandas as pd
results = pd.DataFrame(columns=['number','status','project','description'])

# Extract each row from the parsed input and load into DataFrame
for row in fsm_results:
    results = results.append({
        'number': row[0],
        'status': row[1],
        'project': row[2],
        'description': row[3],
    }, ignore_index=True)

results

number        status project  \
0  0491 570 157        active       A   
1  0491 570 158        active       A   
2  0491 570 159  disconnected       A   
3  0491 570 110        active       B   
4  0491 570 313        active       B   
5  0491 570 737        active       B   
6  0491 571 266  disconnected       C   

                                         description  
0                            used to be in project B  
1                                                     
2                                                     
3  was disconnected on 2019/12/01 activated again...  
4                       previously was 0491 573 087   
5                                                     
6

## Example matching multiple lines of text

Using the phone project script again we can have a look at a different type of text format by using the `--project` option:

In [12]:
! python3 extras/phone_project.py --projects

Project: A
------------------------
Manager    : Ed
Description: Project A

Project: B
------------------------
Manager    : Errol
Description: Project B

Project: C
------------------------
Manager    : Henry
Description: Project C

Project: Update
------------------------
Manager    : Janis
Description: Updating equipment using automation



This is a bit more complex than the example from before where we had only one line at a time in a record. TextFSM will let us parse this, we just need a state with multiple lines.

Lets start by defining the values we need:

- Project: One of "A", "B", "C" or "Update"
- Manager: String
- Description: String that can contain *anything*

In [ ]:
Value ProjectName (A|B|C|Update)
Value ManagerName (\S+)
Value Description (\S.*?)

Now we have to define a state:

In [ ]:
Start
  ^Project:\s ${ProjectName}$$
  ^\-+$$
  ^Manager\s+:\s+${ManagerName}$$
  ^Description\s+:\s+${Description}$$ -> Record

### Running this
First we have to save our template file

In [13]:
%%writefile phone_projects.textfsm
Value ProjectName (A|B|C|Update)
Value ManagerName (\S+)
Value Description (\S.*?)

Start
  ^Project:\s+${ProjectName}$$
  ^\-+$$
  ^Manager\s+:\s+${ManagerName}$$
  ^Description:\s+${Description}$$ -> Record

Writing phone_projects.textfsm


In [14]:
import textfsm

raw_text_data = """Project: A
------------------------
Manager    : Ed
Description: Project A

Project: B
------------------------
Manager    : Errol
Description: Project B

Project: C
------------------------
Manager    : Henry
Description: Project C

Project: Update
------------------------
Manager    : Janis
Description: Updating equipment using automation
"""

# Load in the TextFSM template
with open("phone_projects.textfsm", mode='rt') as template:
    re_table = textfsm.TextFSM(template)
fsm_results = re_table.ParseText(raw_text_data)
fsm_results

[['A', 'Ed', 'Project A'],
 ['B', 'Errol', 'Project B'],
 ['C', 'Henry', 'Project C'],
 ['Update', 'Janis', 'Updating equipment using automation']]

## Practical examples of TextFSM templates

Many common router commands have templates that you can use with them.

Many of these can be found here for common network device commands:

In [ ]:
https://github.com/networktocode/ntc-templates/ 

Some packages like Netmiko will allow you to send commands and will parse the
results for you based on which command is used. In the background this is using
TextFSM to do the parsing.


### Exercise: Parse output from the Cisco `show interfaces description` command

- Parse the data in found in `extras/show_interfaces_output.txt`
- How many are currently "admin down"?
- Extension: Find all the interfaces which contain "upgrade project" in the description

Use the following TextFSM template (found from the ntc-templates project):

In [3]:
%%writefile extras/cisco_iso_show_interfaces_description.textfsm
Value PORT (\S+)
Value STATUS (up|down|admin\s+down)
Value PROTOCOL (up|down)
Value DESCRIP (\S.*?)

Start
  ^Interface\s+Status\s+Protocol\s+Description\s*$$ -> Begin
  ^\s*$$
  # Capture time-stamp if vty line has command time-stamping turned on
  ^Load\s+for\s+
  ^Time\s+source\s+is
  ^. -> Error

Begin
  ^${PORT}\s+${STATUS}\s+${PROTOCOL}(?:\s+${DESCRIP})?\s*$$ -> Record
  ^\s*$$
  ^. -> Error

Writing extras/cisco_iso_show_interfaces_description.textfsm


In [4]:
# %load solutions/parse_cisco_show_interfaces_pandas.py

## Example: analyzing and joining results

A task that will frequently come up is to pull data from multiple devices and then
join those results such that you can do some analysis. Let's look at a way
you can do this using Pandas.

In the example materials from before you can see there is a unique ID in the phone number that is occurring in both data sets. If we wanted to ask what manager was responsible for a phone number we would have to join the results from both the outputs.

In [18]:
import subprocess

raw_data_bytes = subprocess.check_output(
    ['python','extras/phone_project.py'])
summary_raw_data = raw_data_bytes.decode('UTF8')
print(summary_raw_data[:300])

number       status       project  description
0491 570 157 active       A        used to be in project B
0491 570 158 active       A        
0491 570 159 disconnected A        
0491 570 110 active       B        was disconnected on 2019/12/01 activated again 2019/12/02
0491 570 313 active       B  


In [19]:
raw_data_bytes = subprocess.check_output(
    ['python','extras/phone_project.py', '--projects'])
projects_raw_data = raw_data_bytes.decode('UTF8')
print(projects_raw_data[:155])

Project: A
------------------------
Manager    : Ed
Description: Project A

Project: B
------------------------
Manager    : Errol
Description: Project B




### Create the Pandas DataFrames

In [25]:
with open("solutions/phone_info_all.textfsm", mode='rt') as template:
    re_table = textfsm.TextFSM(template)

fsm_results = re_table.ParseText(summary_raw_data)
results_table_data = pd.DataFrame(
    fsm_results,
    columns=['number', 'status', 'project', 'description']
)

results_table_data.head()

number        status project  \
0  0491 570 157        active       A   
1  0491 570 158        active       A   
2  0491 570 159  disconnected       A   
3  0491 570 110        active       B   
4  0491 570 313        active       B   

                                         description  
0                            used to be in project B  
1                                                     
2                                                     
3  was disconnected on 2019/12/01 activated again...  
4                        previously was 0491 573 087

In [26]:
with open("extras/phone_projects.textfsm", mode='rt') as template:
    re_table = textfsm.TextFSM(template)

project_results = re_table.ParseText(projects_raw_data)
projects = pd.DataFrame(
    project_results,
    columns=['project', 'manager', 'description']
)
projects.head()

project manager                          description
0       A      Ed                            Project A
1       B   Errol                            Project B
2       C   Henry                            Project C
3  Update   Janis  Updating equipment using automation

### Merging the DataFrames

You can merge the two dataframes using the Pandas `join` method or its more flexible cousin, `merge`:

In [27]:
merged = results_table_data.merge(
    projects,
    left_on="project", right_on='project')

In [28]:
merged.head()

number        status project  \
0  0491 570 157        active       A   
1  0491 570 158        active       A   
2  0491 570 159  disconnected       A   
3  0491 570 110        active       B   
4  0491 570 313        active       B   

                                       description_x manager description_y  
0                            used to be in project B      Ed     Project A  
1                                                         Ed     Project A  
2                                                         Ed     Project A  
3  was disconnected on 2019/12/01 activated again...   Errol     Project B  
4                        previously was 0491 573 087   Errol     Project B